# Converts a parquet file to CSV file with header using ApacheSpark and Train a Random Forest model with different hyperparameters

In [1]:
%%bash
export version=`python --version |awk '{print $2}' |awk -F"." '{print $1$2}'`

echo $version

if [ $version == '36' ] || [ $version == '37' ]; then
    echo 'Starting installation...'
    pip3 install pyspark==2.4.8 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
elif [ $version == '38' ] || [ $version == '39' ]; then
    pip3 install pyspark==3.1.2 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
else
    echo 'Currently only python 3.6, 3.7 , 3.8 and 3.9 are supported, in case you need a different version please open an issue at https://github.com/IBM/claimed/issues'
    exit -1
fi

37
Starting installation...
Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)


In [1]:
# @param data_dir temporal data storage for local execution
# @param data_csv csv path and file name (default: data.csv)
# @param data_parquet path and parquet file name (default: data.parquet)
# @param master url of master (default: local mode)

In [25]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import shutil
import glob
from pyspark import SparkContext, SparkConf, SQLContext
import os
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark2pmml import PMMLBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import logging
import shutil
import site
import sys
import wget
import re

In [3]:
data_csv = os.environ.get('data_csv', 'data.csv')
data_parquet = os.environ.get('data_parquet', 'data.parquet')
master = os.environ.get('master', "local[*]")
data_dir = os.environ.get('data_dir', '../../data/')

In [8]:
data_parquet = 'data.parquet'
data_csv = 'data.csv'

In [9]:
skip = False
if os.path.exists(data_dir + data_csv):
    skip = True

In [10]:
if not skip:
    sc = SparkContext.getOrCreate(SparkConf().setMaster(master))
    spark = SparkSession.builder.getOrCreate()

In [11]:
if not skip:
    df = spark.read.parquet(data_dir + data_parquet)

In [12]:
if not skip:
    if os.path.exists(data_dir + data_csv):
        shutil.rmtree(data_dir + data_csv)
    df.coalesce(1).write.option("header", "true").csv(data_dir + data_csv)
    file = glob.glob(data_dir + data_csv + '/part-*')
    shutil.move(file[0], data_dir + data_csv + '.tmp')
    shutil.rmtree(data_dir + data_csv)
    shutil.move(data_dir + data_csv + '.tmp', data_dir + data_csv)

In [19]:
df = spark.read.format("csv").option("header", "true").load(data_dir + data_csv)

In [26]:
from pyspark.sql.types import DoubleType
df = df.withColumn("x", df.x.cast(DoubleType()))
df = df.withColumn("y", df.y.cast(DoubleType()))
df = df.withColumn("z", df.z.cast(DoubleType()))

In [27]:
splits = df.randomSplit([0.8, 0.2], seed=1)
df_train = splits[0]
df_test = splits[1]

In [28]:
num_trees = [10, 20]
max_depth = [5, 7]
best_accuracy = 0.0
best_params = None

In [29]:
for trees in num_trees:
    for depth in max_depth:
        # Create the pipeline for Random Forest
        indexer = StringIndexer(inputCol="class", outputCol="label")
        vectorAssembler = VectorAssembler(inputCols=["x", "y", "z"], outputCol="features")
        normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")
        rf = RandomForestClassifier(numTrees=trees, maxDepth=depth, seed=1)

        pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, rf])

In [30]:
        model = pipeline.fit(df_train)

In [31]:
        prediction = model.transform(df_test)

In [32]:
        evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
        accuracy = evaluator.evaluate(prediction)

In [33]:
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = (trees, depth)

In [34]:
print("Best performing hyperparameters:")
print("Number of trees:", best_params[0])
print("Maximum depth:", best_params[1])
print("Accuracy:", best_accuracy)


Best performing hyperparameters:
Number of trees: 20
Maximum depth: 7
Accuracy: 0.5286537064874086
